In [3]:
import vertica_python 
import datetime as dt
#import itertools as it
import numpy as np
import pandas as pd
import os

#### Для работы с данными необходимо перелить данные на одну машину, где все будет агрегироваться

In [57]:
def create_remote_table(query, file_to_save, sep=';'):
    remote_command = "vsql -U dbadmin -F '{sep}' -A -o {file_name} -c '{query};'".format(**{
            'file_name':file_to_save, 
            'query':query,
            'sep':sep,
        })
    os.system('ssh datahub "{}"'.format(remote_command))
    os.system('ssh datahub "sed -i \'$d\' {}"'.format(file_name))
    
    return remote_command


def load_table_from_dh20_to_dh21(path_file_src, path_file_dest, clear_source=False, drop_last_list=True, 
                                      clear_dest_source=False, fast_rsync=True):
    if clear_dest_source:
        os.remove(path_file_dest)
    if fast_rsync:
        rsync_cmd_template = 'rsync -aHAXxv --numeric-ids --delete -e "ssh -T -c arcfour -o Compression=no -x" datahub:{} {}'
    else:
        rsync_cmd_template = 'rsync -aP datahub:{} {}'
        
    os.system(rsync_cmd_template.format(path_file_src, path_file_dest))
    if clear_source:
        os.system('ssh datahub "rm {}"'.format(path_file_src))
    # drop last line because vertica save total number of rows in last line
#     os.system("sed -i '$d' {}".format(path_file_dest))
        
def do_all_thats_stuff(query, path_file_src, path_file_dest, clear_source=False, sep=';'):
    create_remote_table(query, path_file_src, sep=sep)
    load_table_from_dh20_to_dh21(path_file_src, path_file_dest, clear_source)
    
def get_params(table_name):
#     full_table_name = '{}.{}'.format(table_schema, table_name)
    query_to_save = 'select * from {}'.format(table_name)
    src_file = '/home/alfadata/data/{}.csv'.format(table_name)
    dest_file = '/home/alfadata/data/churn/{}.csv'.format(table_name)
    return query_to_save, src_file, dest_file

In [5]:
conn_vert = vertica_python.connect(host = host, port=5433, user=user, password=pass, database=base)
cur_vert = conn_vert.cursor()

### Fetch and transfer data from Vertica to datahub-21 using datahub-20

In [4]:
query_all_churn_tables= """
select distinct(table_name) from COLUMNS
where table_schema='churn' and table_name like 'churn_feature%'
"""

cur_vert.execute(query_all_churn_tables)
data = cur_vert.fetchall()
tables = [record[0] for record in data]
tables

['churn_feature_product_touch',
 'churn_feature_money_withdraw',
 'churn_feature_session_time1',
 'churn_feature_cardtransaction_stran',
 'churn_feature_num_cards',
 'churn_feature_session_time',
 'churn_feature_segments',
 'churn_feature_epayoperation_stran',
 'churn_feature_target',
 'churn_feature_mobile_sessions']

In [4]:
tables = ([
#  'churn_feature_cardtransaction_stran',
 'churn_feature_num_cards'])

In [6]:
tables = ([
#  'churn_feature_cardtransaction_stran',
 'churn_feature_segments'])

In [60]:
for table_name_raw in tables:
    table_name = 'churn.' + table_name_raw
    query_to_save, src_file, dest_file = get_params(table_name)

    start = dt.datetime.now()
    create_remote_table(query_to_save, src_file)
    print('{} is created for {:.1f} seconds'.format(table_name, (dt.datetime.now() - start).total_seconds()))
    load_table_from_dh20_to_dh21(src_file, dest_file, clear_source=False, clear_dest_source=True, fast_rsync=True)
    print('{} is downloaded for total {:.1f} seconds'.format(table_name, (dt.datetime.now() - start).total_seconds()))

churn.churn_feature_segments is created for 7.2 seconds
churn.churn_feature_segments is downloaded for total 30.0 seconds


In [49]:
%%time
df = pd.read_csv('/home/alfadata/data/churn/churn.churn_feature_segments.csv', sep=';')

CPU times: user 6.94 s, sys: 564 ms, total: 7.51 s
Wall time: 7.5 s


In [51]:
df = df.iloc[:-1]

In [26]:
ls

churn_create_vertica_features.ipynb  join_all_data.ipynb
churn_prediction.ipynb               join_all_data.py*
feature_engineering.ipynb            transfer_dh20_to_dh21.ipynb
feature_engineering_sample.ipynb


In [23]:
os.system('touch file')

0

In [25]:
os.remove('/home/alfadata/scripts/aprotopopov/churn/file')

In [5]:
%%time
table_name = 'churn.churn_feature_num_cards'
# query_to_save = 'select * from {} limit 100000'.format(table_name)
# src_file = '/home/alfadata/data/{}.csv'.format(table_name)
# dest_file = '/home/alfadata/data/churn/{}.csv'.format(table_name)
query_to_save, src_file, dest_file = get_params(table_name)

create_remote_table(query_to_save, src_file)
load_table_from_dh20_to_dh21(src_file, dest_file, clear_source=False)
# do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

CPU times: user 23.6 ms, sys: 28.9 ms, total: 52.5 ms
Wall time: 13min 10s


In [ ]:
%%time
table_name = 'oracle.client_sdim'
query_to_save, src_file, dest_file = get_params(table_name)
create_remote_table(query_to_save, src_file)
load_table_from_dh20_to_dh21(src_file, dest_file, clear_source=False)
# do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

In [58]:
%%time
table_name = 'churn.churn_feature_epayoperation_stran'
query_to_save, src_file, dest_file = get_params(table_name)
create_remote_table(query_to_save, src_file)
load_table_from_dh20_to_dh21(src_file, dest_file, clear_source=False)
# do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

CPU times: user 20 µs, sys: 4 µs, total: 24 µs
Wall time: 39.1 µs


In [57]:
%%time
table_name = 'churn.churn_feature_cardtransaction_stran'
query_to_save, src_file, dest_file = get_params(table_name)
do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

CPU times: user 17.9 ms, sys: 21.6 ms, total: 39.5 ms
Wall time: 9min 6s


In [70]:
%%time
table_name = 'churn.churn_feature_mobile_sessions'
query_to_save, src_file, dest_file = get_params(table_name)
do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

CPU times: user 9.54 ms, sys: 21.8 ms, total: 31.4 ms
Wall time: 7min 9s


In [69]:
%%time
table_name = 'churn.churn_feature_target'
query_to_save, src_file, dest_file = get_params(table_name)
do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

CPU times: user 18.2 ms, sys: 27.5 ms, total: 45.7 ms
Wall time: 11min 19s


In [ ]:
# create_remote_table(query_to_save, src_file)
# load_table_from_dh20_to_dh21(src_file, dest_file, clear_source=False)

In [63]:
%%time
table_name = 'churn.churn_feature_money_withdraw'
query_to_save, src_file, dest_file = get_params(table_name)
do_all_thats_stuff(query_to_save, src_file, dest_file, clear_source=False, sep=';')

CPU times: user 2.05 ms, sys: 9.84 ms, total: 11.9 ms
Wall time: 9.75 s
